# Dynamic Bayesian Networks & Markov Chains


Let's start with some basic Markov Chains ...

### Numpy Implementation of a basic Markov Chain

In [1]:
import numpy as np 
from numpy.linalg import matrix_power
from numpy.linalg import inv

In [2]:
# Let's define a transition matrix
P = np.array([[0.9, 0.1],[0.2,0.8]])
print(P)

[[0.9 0.1]
 [0.2 0.8]]


In [3]:
# One, two, three steps ahead
print(P)
print()
print(P.dot(P))
print()
print(P.dot(P).dot(P))

[[0.9 0.1]
 [0.2 0.8]]

[[0.83 0.17]
 [0.34 0.66]]

[[0.781 0.219]
 [0.438 0.562]]


In [4]:
# Use matrix power for this
print(matrix_power(P,3))
print()

[[0.781 0.219]
 [0.438 0.562]]



In [5]:
# Steady state distribution
print(matrix_power(P,3))
print()
print(matrix_power(P,5))
print()
print(matrix_power(P,10))
print()
print(matrix_power(P,20))
print()
print(matrix_power(P,30))
print()
print(matrix_power(P,50))

[[0.781 0.219]
 [0.438 0.562]]

[[0.72269 0.27731]
 [0.55462 0.44538]]

[[0.67608251 0.32391749]
 [0.64783498 0.35216502]]

[[0.66693264 0.33306736]
 [0.66613472 0.33386528]]

[[0.66667418 0.33332582]
 [0.66665164 0.33334836]]

[[0.66666667 0.33333333]
 [0.66666665 0.33333335]]


In [6]:
# Next state distribution ... from an initial state
#q = np.array([0.2, 0.8])
q = np.array([0.0, 1.0])
print(q)
print()
print(q.dot(P))
print()
print(q.dot(matrix_power(P,5)))
print()
print(q.dot(matrix_power(P,15)))
print()
print(q.dot(matrix_power(P,50)))

[0. 1.]

[0.2 0.8]

[0.55462 0.44538]

[0.66350163 0.33649837]

[0.66666665 0.33333335]


### Numpy Implementation of a Markov Chain with an absorbing state

In [7]:
# A Markov Chain with an absorbing state

P = np.array([[0.8, 0.15, 0, 0.05, 0],[0, 0.7, 0.2, 0.1, 0],[0,0,0.95,0,0.05],[0, 0, 0, 1, 0],[0, 0, 0, 0, 1]])
print(P)
print()

Q = P[0:3,0:3]
R = P[0:3,3:5]
print(Q)
print()
print(R)

[[0.8  0.15 0.   0.05 0.  ]
 [0.   0.7  0.2  0.1  0.  ]
 [0.   0.   0.95 0.   0.05]
 [0.   0.   0.   1.   0.  ]
 [0.   0.   0.   0.   1.  ]]

[[0.8  0.15 0.  ]
 [0.   0.7  0.2 ]
 [0.   0.   0.95]]

[[0.05 0.  ]
 [0.1  0.  ]
 [0.   0.05]]


In [8]:
# Average time spent on a transient state
Transient = inv((np.eye(3)-Q))
print(Transient)

[[ 5.          2.5        10.        ]
 [ 0.          3.33333333 13.33333333]
 [ 0.          0.         20.        ]]


In [9]:
# Probability to get in an absorbing state
Absorbing = inv((np.eye(3)-Q)).dot(R)
print(Absorbing)

[[0.5        0.5       ]
 [0.33333333 0.66666667]
 [0.         1.        ]]


In [10]:
# Gambler example

# States are: 3, 4, 2, 1, 5, 0
P = np.array([[  0,  1/3, 2/3,   0,   0,    0],
              [2/3,    0,   0,   0, 1/3,    0],
              [1/3,    0,   0, 2/3,   0,    0],
              [  0,    0, 1/3,   0,   0,  2/3],
              [  0,    0,   0,   0,   1,    0],
              [  0,    0,   0,   0,   0,    1]])
print(P)
Q=P[0:4,0:4]
print(Q)
R=P[0:4,4:6]
print(R)

print()
# Average time spent on a transient state
Transient = inv((np.eye(4)-Q))
print(Transient)
print()

# Probability to get in an absorbing state
Absorbing = inv((np.eye(4)-Q)).dot(R)
print(Absorbing)


[[0.         0.33333333 0.66666667 0.         0.         0.        ]
 [0.66666667 0.         0.         0.         0.33333333 0.        ]
 [0.33333333 0.         0.         0.66666667 0.         0.        ]
 [0.         0.         0.33333333 0.         0.         0.66666667]
 [0.         0.         0.         0.         1.         0.        ]
 [0.         0.         0.         0.         0.         1.        ]]
[[0.         0.33333333 0.66666667 0.        ]
 [0.66666667 0.         0.         0.        ]
 [0.33333333 0.         0.         0.66666667]
 [0.         0.         0.33333333 0.        ]]
[[0.         0.        ]
 [0.33333333 0.        ]
 [0.         0.        ]
 [0.         0.66666667]]

[[2.03225806 0.67741935 1.74193548 1.16129032]
 [1.35483871 1.4516129  1.16129032 0.77419355]
 [0.87096774 0.29032258 2.03225806 1.35483871]
 [0.29032258 0.09677419 0.67741935 1.4516129 ]]

[[0.22580645 0.77419355]
 [0.48387097 0.51612903]
 [0.09677419 0.90322581]
 [0.03225806 0.96774194]]


###  Hidden Markov Models: Umbrella network

In [11]:
# Import libraries
import pgmpy.models
import pgmpy.inference
import numpy as np
from numpy.linalg import matrix_power

In [12]:
# Create a dynamic bayesian network
model = pgmpy.models.DynamicBayesianNetwork()
# Add nodes
model.add_nodes_from(['Weather', 'Umbrella'])
# Print nodes
print('--- Nodes ---')
print(model.nodes())

--- Nodes ---
[<DynamicNode(Weather, 0) at 0x7f9ef0717580>, <DynamicNode(Umbrella, 0) at 0x7f9ef0717c10>]


In [13]:
# Add edges
model.add_edges_from([(('Weather',0), ('Umbrella',0)),
                      (('Weather',0), ('Weather',1)),])
# Print edges
print('--- Edges ---')
print(model.edges())

--- Edges ---
[(<DynamicNode(Weather, 0) at 0x7f9ef0717580>, <DynamicNode(Umbrella, 0) at 0x7f9ef0717e80>), (<DynamicNode(Weather, 0) at 0x7f9ef0717580>, <DynamicNode(Weather, 1) at 0x7f9ef0717a30>), (<DynamicNode(Weather, 1) at 0x7f9ef0717ee0>, <DynamicNode(Umbrella, 1) at 0x7f9ef0717f40>)]


In [14]:
# Print parents
print('--- Parents ---')
print('Umbrella 0: {0}'.format(model.get_parents(('Umbrella', 0))))
print('Weather 0: {0}'.format(model.get_parents(('Weather', 0))))
print('Weather 1: {0}'.format(model.get_parents(('Weather', 1))))
print('Umbrella 1: {0}'.format(model.get_parents(('Umbrella', 1))))

--- Parents ---
Umbrella 0: [<DynamicNode(Weather, 0) at 0x7f9ef0717490>]
Weather 0: []
Weather 1: [<DynamicNode(Weather, 0) at 0x7f9ef0717af0>]
Umbrella 1: [<DynamicNode(Weather, 1) at 0x7f9ef0717ee0>]


In [17]:
# Add probabilities
initial_state_cpd = pgmpy.factors.discrete.TabularCPD(
                                    ('Weather', 0), 2, [[0.3],[0.7]])
umbrella_cpd = pgmpy.factors.discrete.TabularCPD(
                                    ('Umbrella', 0), 2, [[0.8,0.1], 
                                                        [0.2,0.9]], 
                                        evidence=[('Weather', 0)], 
                                        evidence_card=[2])
transition_cpd = pgmpy.factors.discrete.TabularCPD(
                                    ('Weather', 1), 2, [[0.7,0.3], 
                                                       [0.3,0.7]], 
                                     evidence=[('Weather', 0)], 
                                     evidence_card=[2])

# Add conditional probability distributions (cpd:s)
model.add_cpds(initial_state_cpd,umbrella_cpd, transition_cpd)

# This method will automatically re-adjust the cpds and the edges added to the bayesian network.
model.initialize_initial_state()

# Check if the model is valid, throw an exception otherwise
model.check_model()

model.get_cpds()
print('Initial probability distribution, P(Weather(0))')
print(initial_state_cpd)
print()
print('Probability distribution, P(Weather(1) | Weather(0))')
print(transition_cpd)
print()
print('Probability distribution, P(Umbrella(0) | Weather(0))')
print(umbrella_cpd)

Initial probability distribution, P(Weather(0))
+-------------------+-----+
| ('Weather', 0)(0) | 0.3 |
+-------------------+-----+
| ('Weather', 0)(1) | 0.7 |
+-------------------+-----+

Probability distribution, P(Weather(1) | Weather(0))
+-------------------+-------------------+-------------------+
| ('Weather', 0)    | ('Weather', 0)(0) | ('Weather', 0)(1) |
+-------------------+-------------------+-------------------+
| ('Weather', 1)(0) | 0.7               | 0.3               |
+-------------------+-------------------+-------------------+
| ('Weather', 1)(1) | 0.3               | 0.7               |
+-------------------+-------------------+-------------------+

Probability distribution, P(Umbrella(0) | Weather(0))
+--------------------+-------------------+-------------------+
| ('Weather', 0)     | ('Weather', 0)(0) | ('Weather', 0)(1) |
+--------------------+-------------------+-------------------+
| ('Umbrella', 0)(0) | 0.8               | 0.1               |
+----------------

In [20]:
# Make Weather prediction via forward inference
# Check here for forward inference and backward inference
#

map = {0: 'Sunny', 1: 'Rainy' }
dbn_inf = pgmpy.inference.DBNInference(model)

# predict 1 step with forward inference
start_slice=0
end_slice = 1
result = dbn_inf.forward_inference([('Weather', end_slice)], {('Weather', start_slice):0})
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0}) | Weather({1}): Sunny): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

# predict 2 step with forward inference
start_slice=0
end_slice = 2
result = dbn_inf.forward_inference([('Weather', end_slice)], {('Weather', start_slice):0})
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0}) | Weather({1}): Sunny): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

P=np.array([[0.7,0.3],[0.3, 0.7]])
print()
print(P)
print()
print(P.dot(P))
print()

# predict 3 step with forward inference no initial state
start_slice=0
end_slice = 3
result = dbn_inf.forward_inference([('Weather', end_slice)])
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0})): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

# predict 3 step with forward inference
start_slice=0
end_slice = 3
result = dbn_inf.forward_inference([('Weather', end_slice)], {('Weather', start_slice):0})
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0}) | Weather({1}): Sunny): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

# predict 3 step with forward inference
start_slice=0
end_slice = 3
result = dbn_inf.forward_inference([('Weather', end_slice)], {('Weather', start_slice):1})
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0}) | Weather({1}): Rainy): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

Prediction (Weather(1) | Weather(0): Sunny): Sunny (70.0 %)
[0.7 0.3]
Prediction (Weather(2) | Weather(0): Sunny): Sunny (57.99999999999999 %)
[0.58 0.42]

[[0.7 0.3]
 [0.3 0.7]]

[[0.58 0.42]
 [0.42 0.58]]

Prediction (Weather(3)): Rainy (51.28 %)
[0.4872 0.5128]
Prediction (Weather(3) | Weather(0): Sunny): Sunny (53.2 %)
[0.532 0.468]
Prediction (Weather(3) | Weather(0): Rainy): Rainy (53.2 %)
[0.468 0.532]


In [21]:
# Make Weather prediction via backward inference
map = {0: 'Sunny', 1: 'Rainy' }
dbn_inf = pgmpy.inference.DBNInference(model)

# predict 1 step with backward inference
start_slice=0
end_slice = 1
result = dbn_inf.backward_inference([('Weather', end_slice)], {('Weather', start_slice):0})
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0}) | Weather({1}): Sunny): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

# predict 2 step with backward inference
start_slice=0
end_slice = 2
result = dbn_inf.backward_inference([('Weather', end_slice)], {('Weather', start_slice):0})
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0}) | Weather({1}): Sunny): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

# predict 2 step with backward inference
start_slice=0
end_slice = 3
result = dbn_inf.backward_inference([('Weather', end_slice)], {('Weather', start_slice):0})
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0}) | Weather({1}): Sunny): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

Prediction (Weather(1) | Weather(0): Sunny): Rainy (50.000000000000014 %)
[0.5 0.5]
Prediction (Weather(2) | Weather(0): Sunny): Sunny (57.99999999999999 %)
[0.58 0.42]
Prediction (Weather(3) | Weather(0): Sunny): Sunny (53.2 %)
[0.532 0.468]


In [22]:
# What about the steady state distribution?

P = np.array([[0.7,0.3],[0.3,0.7]])
print('Steady state distribution: ')
print()
print(matrix_power(P,20))
print()

# predict 2 step with forward inference
end_slice = 20
result = dbn_inf.forward_inference([('Weather', end_slice)])
print(result[('Weather', end_slice)].values)

# predict 2 step with forward inference
end_slice = 20
result = dbn_inf.forward_inference([('Weather', end_slice)],{('Weather',0):0})
print(result[('Weather', end_slice)].values)

# predict 2 step with forward inference
end_slice = 20
result = dbn_inf.forward_inference([('Weather', end_slice)],{('Weather',0):1})
print(result[('Weather', end_slice)].values)

#arr = result[('Weather', end_slice)].values
#print('Prediction (Weather({0}) | Weather({1}): Sunny): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))

Steady state distribution: 

[[0.50000001 0.49999999]
 [0.49999999 0.50000001]]

[0.5 0.5]
[0.50000001 0.49999999]
[0.49999999 0.50000001]


In [23]:
# Make Weather prediction via forward inference, with and without evidence
map = {0: 'Sunny', 1: 'Rainy' }
dbn_inf = pgmpy.inference.DBNInference(model)
start_slice=0
end_slice = 2

# predict 2 step with forward inference
result = dbn_inf.forward_inference([('Weather', end_slice)])
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0})): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

# predict 2 step with forward inference
result = dbn_inf.forward_inference([('Weather', end_slice)], evidence = {('Weather',0):0})
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0}) | Weather({1}): Sunny): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

# predict 2 step with forward inference
result = dbn_inf.forward_inference([('Weather', end_slice)], evidence = {('Weather',0):1})
arr = result[('Weather', end_slice)].values
print('Prediction (Weather({0}) | Weather({1}): Rainy): {2} ({3} %)'.format(end_slice, start_slice, map[np.argmax(arr)], np.max(arr) * 100))
print(arr)

Prediction (Weather(2)): Rainy (53.19999999999999 %)
[0.468 0.532]
Prediction (Weather(2) | Weather(0): Sunny): Sunny (57.99999999999999 %)
[0.58 0.42]
Prediction (Weather(2) | Weather(0): Rainy): Rainy (57.99999999999999 %)
[0.42 0.58]


In [24]:
result = dbn_inf.forward_inference([('Weather', 1)], {('Umbrella', 0):0})
arr = result[('Weather', 1)].values
print('Prediction (Weather(1) | Umbrella(0): False): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
print()

result = dbn_inf.backward_inference([('Weather', 1)], {('Umbrella', 1):0})
arr = result[('Weather', 1)].values
print('Prediction (Weather(1) | Umbrella(1): False): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
print()

result = dbn_inf.backward_inference([('Weather', 0)], {('Umbrella', 1):0})
arr = result[('Weather', 0)].values
print('Prediction (Weather(0) | Umbrella(1): False): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
print()

Prediction (Weather(1) | Umbrella(0): False): Sunny (60.96774193548388 %)

Prediction (Weather(1) | Umbrella(1): False): Sunny (85.27918781725889 %)

Prediction (Weather(0) | Umbrella(1): False): Rainy (55.07614213197969 %)



In [29]:
model.initialize_initial_state()

result = dbn_inf.backward_inference([('Weather', 1)], {('Weather', 0):0})
arr = result[('Weather', 1)].values
print('Prediction (Weather(1) | Weather(0): Sunny): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
print()

result = dbn_inf.backward_inference([('Weather', 1)], {('Umbrella', 0):0})
arr = result[('Weather', 1)].values
print('Prediction (Weather(1) | Umbrella(0): False): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
print()

result = dbn_inf.forward_inference([('Weather', 1)], {('Umbrella', 0):0, ('Weather', 0):0})
arr = result[('Weather', 1)].values
print('Prediction (Weather(1) | Weather(0): Sunny, Umbrella(0): False): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
print()

result = dbn_inf.forward_inference([('Weather', 1)], {('Umbrella', 1):0, ('Weather', 0):0})
arr = result[('Weather', 1)].values
print('Prediction (Weather(1) | Weather(0): Sunny, Umbrella(1): False): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
print()

result = dbn_inf.forward_inference([('Weather', 1)], {('Umbrella', 1):0, ('Umbrella', 0):0})
arr = result[('Weather', 1)].values
print('Prediction (Weather(1) | Umbrella(0): False, Umbrella(1): False): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
print()

result = dbn_inf.forward_inference([('Weather', 1)], {('Umbrella', 1):0})
arr = result[('Weather', 1)].values
print('Prediction (Weather(1) | Umbrella(1): False): {0} ({1} %)'.format(map[np.argmax(arr)], np.max(arr) * 100))
print()

Prediction (Weather(1) | Weather(0): Sunny): Rainy (50.000000000000014 %)

Prediction (Weather(1) | Umbrella(0): False): Sunny (60.96774193548388 %)

Prediction (Weather(1) | Weather(0): Sunny, Umbrella(0): False): Sunny (70.0 %)

Prediction (Weather(1) | Weather(0): Sunny, Umbrella(1): False): Sunny (94.91525423728812 %)

Prediction (Weather(1) | Umbrella(0): False, Umbrella(1): False): Sunny (92.59032455603185 %)

Prediction (Weather(1) | Umbrella(1): False): Sunny (85.27918781725889 %)

